In [ ]:
! pip install -qU langchain chromadb huggingface_hub sentence_transformers pypdf openai tiktoken langchain-community

In [ ]:
! pip install llama-cpp-python

Loading Data

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings, OpenAIEmbeddings, HuggingFaceBgeEmbeddings
from google.colab import userdata
from langchain.vectorstores import Chroma
import chromadb
import os
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.document_transformers import (
    EmbeddingsClusteringFilter,
    EmbeddingsRedundantFilter,
)
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_transformers import LongContextReorder
from re import search
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA


In [ ]:
harrypotter_loader = PyPDFLoader("/content/harry_potter_book.pdf")

In [ ]:
harrypotter_documents = harrypotter_loader.load()

In [ ]:
print(len(harrypotter_documents))

In [ ]:
got_loader = PyPDFLoader("/content/got_book.pdf")

In [ ]:
got_documents = got_loader.load()

In [ ]:
print(len(got_documents))

Creating Text Splitter for Chunking

In [ ]:
text_splitter =RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [ ]:
harrypotter_text = text_splitter.split_documents(harrypotter_documents)

In [ ]:
got_text = text_splitter.split_documents(got_documents)

In [ ]:
print(len(harrypotter_text)), print(len(got_text))

Load Embedding Model to Conver the Data into Vector

In [ ]:
hf_embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
hf_bge_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-Large-en")

In [ ]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [ ]:
openai_embeddings = OpenAIEmbeddings(openai_api_type=OPENAI_API_KEY)

Ingest Data into Chroma Database

In [ ]:
os.getcwd()

In [ ]:
CURRENT_DIR = os.path.dirname(os.path.abspath("."))

In [ ]:
DB_DIR = os.path.join(CURRENT_DIR, "/content/db")

In [ ]:
client_settings = chromadb.config.Settings(
    is_persistent = True,
    persist_directory=DB_DIR,
    anonymized_telemetry=False,
)

In [ ]:
harrypotter_vectorstore = Chroma.from_documents(
    harrypotter_text,
    hf_bge_embeddings,
    client_settings=client_settings,
    collection_name="Harrypotter",
    collection_metadata={"hnsw":"cosine"},
    persist_directory="/store/harrypotter"
)

In [ ]:
got_vectorstore = Chroma.from_documents(
    got_text,
    hf_bge_embeddings,
    client_settings=client_settings,
    collection_name="got",
    collection_metadata={"hnsw":"cosine"},
    persist_directory="/store/got"
)

Now Create Retriever

In [ ]:
harrypotter_retriever = harrypotter_vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":5, "include_metadata":True})

In [ ]:
got_retriever = got_vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":5, "include_metadata":True})

Merge both Retriever 
(It is also called lord of retriever(LOTR))

In [ ]:
lotr = MergerRetriever(retrivers=[harrypotter_retriever, got_retriever])

In [ ]:
for chunks in lotr.get_relevant_documents("Who was jon snow?"):
    print(chunks.page_content)

In [ ]:
for chunks in lotr.get_relevant_documents("Who is Harry potter?"):
    print(chunks.page_content)

Now after understanding step by step, time to create a pipeline for LLM

In [ ]:
_filter = EmbeddingsRedundantFilter(embeddings=hf_bge_embeddings)
reordering = LongContextReorder()
pipeline = DocumentCompressorPipeline(transformers=[_filter, reordering])
compression_retriever_reordered = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr, search_kwargs={"k":3, "include_metadata":True}
)

In [ ]:
docs = compression_retriever_reordered.get_relevant_documents("What is esops?")
print(len(docs))

print(docs[0].page_content)

Loading Model from HuggingFace

In [ ]:
llms = LlamaCpp(
    streaming=True,
    model_path="/content/drive/MyDrive/zephyr-7b-beta.Q4_K_M.gguf",
    max_tokens=1500,
    temperature=0.75,
    top_p=1,
    gpu_layers=0,
    stream=True,
    verbose=True,
    n_threads=int(os.cpu_count()/2),
    n_ctx=4096
)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llms,
    chain_type="stuff",
    retriver=compression_retriever_reordered,
    return_source_documents = True
)

In [ ]:
query = "Who is jon Snow?"
results = qa(query)
print(results["result"])
print(results["source_documents"])

In [ ]:
results_1 = qa("Who is Harry Potter?")
print(results_1["result"])

print(results_1["source_documents"])

for source in results_1["source_documents"]:
    print(source.metadata)

In [ ]:
results_2 = qa.invoke("How does Jon Snow's relationship with Stark family?")

In [ ]:
results_2

In [ ]:
print(results_2["result"])

In [ ]:
print(results_2["source_documents"])

In [ ]:
for source in results_2["source_documents"]:
    print(source.metadata)